<a href="https://colab.research.google.com/github/nyoons/ESAA/blob/main/2022_10_31_%EA%B3%BC%EC%A0%9C_%ED%8C%8C%EC%9D%B4%EC%8D%AC_%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D_%EC%99%84%EB%B2%BD%EA%B0%80%EC%9D%B4%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ch 03. 평가
머신러닝의 프로세스는 데이터 가공-모델 학습과 예측-평가. 성능평가지표는 모델이 분류인지 회귀인지에 따라 여러 종류로 나뉜다. 특히 분류의 성능평가지표로는 정확도, 오차행렬, 정밀도, 재현율, F1 스코어, ROC AUC 등이 있다. 이진/멀티분류에 모두 적용할 수 있지만 이진분류에서 더욱 강조하는 지표.
>**01.정확도**

=예측 결과가 동일한 데이터 건수 / 전체 예측 데이터 건수
<br/>데이터의 구성에 따라 모델의 성능을 왜곡할 수 있어서 수치 하나로는 평가하지 않음. 어떻게 왜곡?
<br/>앞의 타이타닉 고려. ml알고리즘 적용 후 예측 정확도는 보통 80퍼대였지만, 탑승객이 남자인 경우보다 여자인 경우 생존 확률이 높았으니 별다른 알고리즘 없이 무조건 여자인 경우 생존, 남자인 경우 사망으로 예측해도 비슷한 수치가 나올수도. 성별 조건 하나만으로 결정하는 별거 아닌 알고리즘도 정확도가 높아짐.
<br/>BaseEstimator 받아 아무런 학습 없이 성별에 따라 예측하는 단순 classifier 생성해보자

In [ ]:
from sklearn.base import BaseEstimator

class MyDummyClassifier(BaseEstimator):
  #fit메서드는 아무것도 학습하지 않게
  def fit(self, X, y=None):
    pass
  #predict() 메서드는 단순히 sex 피처가 1이면 0, 아니면 1로 예측
  def predict(self, X):
    pred=np.zeros((X.shape[0],1))
    for i in range(X.shape[0]) :
      if X['Sex'].iloc[i]==1:
        pred[i]=0
      else :
        pred[i]=1

    return pred

생성된 MyDummyClassifier 이용해 타이타닉 생존자 예측 해보자

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

#원본 데이터 재로딩, 데이터 가공, 학습 데이터/테스트 데이터로 분할
titanic_df=pd.read_csv('titanic_train.csv')
y_titanic_df=titanic_df['Survived']
X_titanic_df=titanic_df.drop('Survived', axis=1)
X_titanic_df=transform_features(X_titanic_df)
X_train, X_test, y_train, y_test=train_test_split(X_titanic_df, y_titanic_df, test_size=0.2, random_state=0)

#위에서 생선한 Dummy Classifier 이용해 학습/예측/평가 수행
myclf=MyDummyClassifier()
myclf.fit(X_train, y_train)

mypredictions=myclf.predict(X_test)
print('Dummy Classfier의 정확도는: {0:.4f}'.format(accuracy_score(y_test, mypredictions)))

NameError: ignored

이렇게만 해도 정확도가 0.7877. 정확도는 불균형한 레이블 값 분포에서 적합하지 않음.
<br/>MNIST로도 보자. 7만 True, 아닌건 False로 변환해 이진 분류로 바꿔 예측하자.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

class MyDummyClassifier(BaseEstimator):
  def fit(self, X, y):
    pass

  #입력값으로 들어오는 X 데이터 세트 크기만큼 모두 0값으로 만들어 반환
  def predict(self, X):
      return np.zeros( (len(X), 1), dtype=bool)

#사이킷런의 내장 데이터 세트인 load_digits 이용해 MNIST 데이터 로딩
digits=load_digits()

#digits 번호가 7이면 TRUE 이고 이를 astype(int)로 1로 변환, 7 아니면 False 이고 0으로 변환.
y=(digits.target==7).astype(int)
X_train, X_test, y_train, y_test=train_test_split(digits.data, y, random_state=11)


NameError: ignored

불균형한 데이터로 생성한 y_test의 분포 확인, 예측/평가 수행

In [1]:
#불균형한 레이블 데이터 분포 확인
print('레이블 테스트 세트 크기 :', y_test.shape)
print('테스트 세트 레이블 0과 1의 분포도 : ')
print(pd.Series(y_test).value_counts())

#Dummy Classifier로 학습/예측 정확도 평가
fakeclf=MyFakeClassifier()
fakeclf.fit(X_train, y_train)
fakepred=fakeclf.predict(X_test)
print('모든 예측을 0으로 하여도 정확도는:{:.3f}'.format(accuracu_score(y_test,fakepred)))

NameError: ignored

이렇게만 해도 정확도가 90퍼. 정확도는 불균형한 레이블 데이터 세트에서는 사용하면 안됨.

>**2. 오차 행렬**

학습된 분류 모델이 예측을 수행하며 얼마나 헷갈리고 있는지 보여주는 지표. TN, FP, FN, TP를 생각하자.
<br/>사이킷런은 오차 행렬을 구하기 위해 confusion_matrix() 제공

In [2]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, fakepred)


NameError: ignored

ndarray 형태로 출력. TP, TN, FN, FP를 보면 정확도가 왜 오류를 일으키는지 알 수 있다.

> **3. 정밀도와 재현율**

positive 데이터 세트의 예측 성능에 초점을 맞춘 평가 지표. 앞에서 만든 myfakeclf는 positive로 예측한 TP 값이 하나도 없어서 정밀도와 재현율이 모두 0.
<br/>정밀도=TP/(FP+TP)
<br/>재현율=TP/(FN+TP) = 민감도, TPR
<br/>재현율이 중요 지표인 경우는 실제 positive 양성 데이터를 negative로 잘못 판단하게 되면 업무상 큰 영향이 발생하는 경우. 보험사기나 금융사기 적발 모델, 암 판단 모델 등.
<br/>정밀도가 중요한 지표인 경우는 스패메일 여부를 판단하는 모델. 실제 negative 음성인 데이터 예측을 positive 양성으로 잘못 판단하면 업무상 큰 영향이 발생하는 경우.
<br/>사이킷런은 정밀도 계산을 위해 precision_score(), 재현율 계산을 위해 recall_score() 제공

In [3]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
def get_clf_eval(y_test, pred):
  confusion=confusion_matrix(y_test, pred)
  accuracy=precision_score(y_test, pred)
  precision=precision_score(y_test, pred)
  recall=recall_score(y_test, pred)
  print('오차 행렬')
  print(confusion)
  print('정확도:{0:.4f}, 재현율:{2:.4f}'.format(accuracy, precision, recall))


In [4]:
#이제 로지스틱 회귀 기반으로 타이타닉 예측하고 평가 수행.
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

#원본 데이터 재로딩, 데이터 가공, 학습 데이터/테스트 데이터 분할
titanic_df=pd.read_csv('titanic_train.csv')
y_titanic_df=titanic_df['Survived']
X_titanic_df=titanic_df.drop('Survived', axis=1)
X_titanic_df=transform_features(X_titanic_df)

X_train, X_test, y_train, y_test=train_test_split(X_titanic_df, y_titanic_df, test_size=0.20, random_state=11)

lr_clf=LogisticRegression()
lr_clf.fit(X_train, y_train)
pred=lr_clf.predict(X_test)
get_clf_eval(y_test,pred)

FileNotFoundError: ignored

재현율이 좀 낮게 나왔다. 강화할 방법?

###정밀도/재현율 트레이드오프

ㅀ
